In [7]:
%load_ext autoreload
%autoreload 2

In [14]:
from reason.core import AbstractTerm, Variable
from reason.core.transform import explode_over_conjunctions
from reason.vampire.translator import to_fof
from reason.parser import Parser
parser = Parser()

In [9]:
import logging
import sys

# Configure the logging
logging.basicConfig(
    level=logging.DEBUG,  # Set the logging level to DEBUG
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',  # Log format
    handlers=[
        logging.StreamHandler(sys.stdout)  # Log to standard output
    ]
)

In [10]:
print(parser.ogc.create_lark_code())



logic_simple: logic_level_8

logic_level_0: abstract_term | "(" logic_simple ")" -> brk | "{" logic_simple_list "}" -> conj_formula 

OP2_1.1: "∩"
OP2_2.1: "∪"
OP2_3.1: "∈"| "="| "⊂"
OP1_4.1: "~"
OP2_5.1: "and"| "∧"
OP2_6.1: "or"| "∨"
OP2_7.1: "→"| "⇒"| "⟷"| "⇔"
OP1_8.1: "∀"| "∃"

logic_level_1: logic_level_1 OP2_1 logic_level_0 -> _op2 | logic_level_0
logic_level_2: logic_level_2 OP2_2 logic_level_1 -> _op2 | logic_level_1
logic_level_3: logic_level_3 OP2_3 logic_level_2 -> _op2 | logic_level_2
logic_level_4: OP1_4 logic_level_3 -> _op1 | logic_level_3
logic_level_5: logic_level_5 OP2_5 logic_level_4 -> _op2 | logic_level_4
logic_level_6: logic_level_6 OP2_6 logic_level_5 -> _op2 | logic_level_5
logic_level_7: logic_level_7 OP2_7 logic_level_6 -> _op2 | logic_level_6
logic_level_8: OP1_8 "(" abstract_term_list ")" logic_level_8 -> _op_quant | logic_level_7




In [20]:
from reason.core.theory import Theory
from reason.vampire import Vampire
vampire_prover = Vampire(verbose=False)
ZFC = Theory(parser, vampire_prover)

ZFC.add_const("∅")

ZFC.add_axiom("∀(x, y) x = y ⟷ (∀(z) z ∈ x ⟷ z ∈ y)", name="a0")
ZFC.add_axiom("empty(e) ⟷ (∀(x) ~(x ∈ e))", name="d1")
ZFC.add_axiom("empty(∅)", name="a1")
ZFC.add_axiom("∀(x, z) z ∈ s(x) ⟷ z = x", name="a3")
ZFC.add_axiom("∀(x, y, z) z ∈ x ∪ y ⟷ z ∈ x ∨ z ∈ y", name='a4')
ZFC.add_axiom("∀(x, y, z) z ∈ x ∩ y ⟷ z ∈ x ∧ z ∈ y", name='a5')
ZFC.add_axiom("∀(x, y) x ⊂ y ⟷ (∀(z) z ∈ x → z ∈ y)", name='d2')

ZFC.add_formula("∀(z) z ∈ a ∩ (b ∪ c) → z ∈ (a ∩ b) ∪ (a ∩ c)", name='t1', type="theorem")
ZFC.add_formula("∀(z) z ∈ (a ∩ b) ∪ (a ∩ c) → z ∈ a ∩ (b ∪ c)", name='t2', type="theorem")

# ZFC.add_formula("(x = z ∧ z = y) → ((((∀(k) k ∈ x ⟷ k ∈ z) ∧ (∀(k) k ∈ z ⟷ k ∈ y) → (∀(k) (k ∈ x ⟷ k ∈ y)))) → (x = y))", 
#                 name="t3",
#                 type="theorem")

# ZFC.add_formula("~(s(x) ∩ s(y) = ∅) → (∃(z) z ∈ s(x) ∧ z ∈ s(y) ∧ z = x ∧ z = y)", 
#                 name="t4",
#                 type="theorem")

In [ ]:
from reason.vampire.translator import to_fof

In [15]:
formula = parser("""∀(x, y){
        ~(s(x) ∩ s(y) = ∅);
        (∃(z) z ∈ s(x) ∧ z ∈ s(y))}""")

In [16]:
explode_over_conjunctions(formula)

[FORALL(x, FORALL(y, CONJUNCTION(NEG(EQ(INTERSECT(s(x), s(y)), ∅))))),
 FORALL(x, FORALL(y, CONJUNCTION(NEG(EQ(INTERSECT(s(x), s(y)), ∅)), EXISTS(z, AND(IN(z, s(x)), IN(z, s(y)))))))]

In [ ]:
ZFC("~(x ∈ y) ∧ (x ∈ y)")

In [3]:
ZFC("~(s(x) ∩ s(y) = ∅) → x = y")

True

In [28]:
ZFC("""(∀(x, y, z, k)
~(s(x) ∩ s(y) = ∅) and
z ∈ s(x) ∧ z ∈ s(y) and
z = x ∧ z = y and
(k ∈ z ⟷ k ∈ x) ∧ (k ∈ z ⟷ k ∈ y) and
(k ∈ x ⟷ k ∈ y) and
x = y        
) → ( ∀(x, y) ~(s(x) ∩ s(y) = ∅) → x = y )""")

True

In [30]:
ZFC("""(
~(s(x) ∩ s(y) = ∅) and
z ∈ s(x) ∧ z ∈ s(y) and
z = x ∧ z = y and
(k ∈ z ⟷ k ∈ x) ∧ (k ∈ z ⟷ k ∈ y) and
(k ∈ x ⟷ k ∈ y) and
x = y        
) → ( ~(s(x) ∩ s(y) = ∅) → x = y )""")

True

In [26]:
ZFC(
  """
  (~(s(x) ∩ s(y) = ∅)) → ((∃(z) (z ∈ s(x) ∧ z ∈ s(y)) → (∃(z) ∀(k) (k ∈ z ⟷ k ∈ x) ∧ (k ∈ z ⟷ k ∈ y))))
  """
)

True

In [23]:
ZFC("""(~(s(x) ∩ s(y) = ∅)) → (
~(s(x) ∩ s(y) = ∅) and
(∃(z) (z ∈ s(x) ∧ z ∈ s(y) and
z = x ∧ z = y and
(∀(k)((k ∈ z ⟷ k ∈ x) ∧ (k ∈ z ⟷ k ∈ y) and
(k ∈ x ⟷ k ∈ y) and
x = y        
)))))""")

True

In [12]:
ZFC("s(x) = s(y) → x = y")

True

In [11]:
ZFC("a ∩ (b ∪ c) = (a ∩ b) ∪ (a ∩ c)")

2025-01-25 12:28:24,323 - root - DEBUG - % input_syntax=auto not supported for standard input parsing, switching to tptp.
% Refutation found. Thanks to Tanya!
% SZS status Theorem for 
% SZS output start Proof for 
1. ! [X0] : ! [X1] : (f_EQ(X0,X1) <=> ! [X2] : (f_IN(X2,X0) <=> f_IN(X2,X1))) [input(axiom)]
8. ! [X2] : (f_IN(X2,f_INTERSECT(X4,f_UNION(X5,X6))) => f_IN(X2,f_UNION(f_INTERSECT(X4,X5),f_INTERSECT(X4,X6)))) [input(assumption)]
9. ! [X2] : (f_IN(X2,f_UNION(f_INTERSECT(X4,X5),f_INTERSECT(X4,X6))) => f_IN(X2,f_INTERSECT(X4,f_UNION(X5,X6)))) [input(assumption)]
10. f_EQ(f_INTERSECT(X4,f_UNION(X5,X6)),f_UNION(f_INTERSECT(X4,X5),f_INTERSECT(X4,X6))) [input(conjecture)]
11. ~! [X6,X4,X5] : f_EQ(f_INTERSECT(X4,f_UNION(X5,X6)),f_UNION(f_INTERSECT(X4,X5),f_INTERSECT(X4,X6))) [negated conjecture 10]
12. ! [X0,X1] : (f_EQ(X0,X1) <=> ! [X2] : (f_IN(X2,X0) <=> f_IN(X2,X1))) [flattening 1]
20. ! [X0] : (f_IN(X0,f_INTERSECT(X1,f_UNION(X2,X3))) => f_IN(X0,f_UNION(f_INTERSECT(X1,X2),f_INTERSEC

True

In [8]:
ZFC("∀(x, y) empty(x) ∧ empty(y) → x = y")

2025-01-25 11:55:41,474 - root - DEBUG - % input_syntax=auto not supported for standard input parsing, switching to tptp.
% Refutation found. Thanks to Tanya!
% SZS status Theorem for 
% SZS output start Proof for 
1. ! [X0] : ! [X1] : (f_EQ(X0,X1) <=> ! [X2] : (f_IN(X2,X0) <=> f_IN(X2,X1))) [input(axiom)]
2. f_empty(X3) <=> ! [X0] : ~f_IN(X0,X3) [input(axiom)]
8. ! [X0] : ! [X1] : ((f_empty(X1) & f_empty(X0)) => f_EQ(X0,X1)) [input(conjecture)]
9. ~! [X0] : ! [X1] : ((f_empty(X1) & f_empty(X0)) => f_EQ(X0,X1)) [negated conjecture 8]
10. ! [X0,X1] : (f_EQ(X0,X1) <=> ! [X2] : (f_IN(X2,X0) <=> f_IN(X2,X1))) [flattening 1]
11. f_empty(X0) <=> ! [X1] : ~f_IN(X1,X0) [rectify 2]
12. ! [X0] : (f_empty(X0) <=> ! [X1] : ~f_IN(X1,X0)) [closure 11]
18. ~! [X0,X1] : ((f_empty(X1) & f_empty(X0)) => f_EQ(X0,X1)) [flattening 9]
19. ! [X0] : (f_empty(X0) => ! [X1] : ~f_IN(X1,X0)) [unused predicate definition removal 12]
20. ! [X0] : (! [X1] : ~f_IN(X1,X0) | ~f_empty(X0)) [ennf transformation 19]
21. ?

True

In [52]:
ZFC("empty(x) → x = ∅")

True

In [53]:
ZFC("s(x) = s(y) → x = y")

True

In [56]:
ZFC("∀(x, y, z) z = s(x) ∪ s(y) → x ∈ z ∧ y ∈ z")

True

In [7]:
ZFC("~(s(x) ∩ s(y) = ∅) → (∃(z) z ∈ s(x) ∧ z ∈ s(y) ∧ z = x ∧ z = y)")

True

In [31]:
ZFC("(z = x ∧ z = y) → x = y")

True

In [9]:
ZFC("~(s(x) ∩ s(y) = ∅) → x = y")

True

In [28]:
ZFC("~(s(x) ∩ s(y) = ∅) → (∃(z) z ∈ s(x) ∧ z ∈ s(y))")

True

In [47]:
ZFC("∀(x, y, z) z ∈ s(x) ∧ z ∈ s(y) → z = x ∧ z = y")

True

In [ ]:
ZFC("∀(x, y, z) x = z ∧ z = y → x = y")

In [25]:
ZFC("(x = z ∧ z = y) → ((((∀(k) k ∈ x ⟷ k ∈ z) ∧ (∀(k) k ∈ z ⟷ k ∈ y) → (∀(k) (k ∈ x ⟷ k ∈ y)))) → x = y)")

True